In [22]:
import os
import torch
import imageio.v2 as imageio
import torchvision

from torch.utils.data import Dataset

from torchvision import transforms

from sklearn.model_selection import train_test_split

from PIL import Image

import torchvision.transforms as T

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import warnings
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T


raw_data_train = r'C:\Users\tariq\Desktop\Tariq\PNW\Spring 2024\ITS 365\CIFAR10_COMP\CIFAR10-Comp-master\train'

dataset_train = []
labels_train  = []
targets_train = []


for folder in os.listdir( raw_data_train ):
    ## print(folder)
    for image in os.listdir( os.path.join(raw_data_train, folder) ):
        if folder not in labels_train:
            labels_train.append( folder )
        targets_train.append(  labels_train.index(folder)  )
        img_arr = imageio.imread(  os.path.join(raw_data_train, folder, image), pilmode="RGB"  )
        
        img = torch.from_numpy( img_arr ).permute( 2, 0, 1 ).float()
        
        img /= 255
        dataset_train.append(img)

data_train    = torch.stack( dataset_train )
targets_train = torch.Tensor(  targets_train  ).type(   torch.LongTensor   )

torch.save(   (data_train, targets_train, labels_train), "InClass_CIFAR10_data"     )

## data1, targets1, labels1 = torch.load("InClass_CIFAR10_data")


img_tr = data_train[46000]

X_train = data_train  
y_train = targets_train


transform = T.ToPILImage()


img = transform(  img_tr  )

y_train_np = targets_train.numpy() 

the_set = np.unique(  y_train_np  )

CIFAR_train_list = [  ( X_train[i],  y_train[i].item() )  for i in range( X_train.shape[0]   )  ]

batch_size = 10

train_dl = torch.utils.data.DataLoader( CIFAR_train_list, batch_size=batch_size, shuffle=True  )

class MLP_net(nn.Module):
    ## init the class
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear( 32*32*3 ,20)
        self.act1    = nn.Sigmoid()
        self.linear2 = nn.Linear(20 , 10)
        self.act2    = nn.Softmax(dim=1)
        
    ## perform inference
    def forward(self, x):
        
        x            = self.linear1(x)
        x            = self.act1(x)
        x            = self.linear2(x)
        y_pred       = self.act2(x)
        
        return y_pred
    
def training_loop(N_Epochs, model, loss_fn, opt):
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            xb = xb.view((16, -1))
            y_pred = model(xb)
            loss = loss_fn(y_pred, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch % 5 == 0:
            print(f"Epoch {epoch}, loss = {loss:.4f}")

N_Epochs      = 100
learning_rate = 0.001

model = MLP_net()

opt = torch.optim.Adam(  model.parameters(), lr=learning_rate  )

loss_fn = nn.CrossEntropyLoss()

def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('F1-measure: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

with torch.no_grad():
    highest_accuracy = 0.0
    for x_real, y_real in train_dl:
        batch_size = x_real.shape[0]
        x_real = x_real.view((batch_size, -1))
        y_pred = model(x_real)
        _, indeces = torch.max(y_pred, dim=1)
        preds = indeces.numpy()
        accuracy = accuracy_score(y_real.numpy(), preds)
        highest_accuracy = max(highest_accuracy, accuracy)

    print(f"Highest accuracy: {highest_accuracy:.4f}")
    

Highest accuracy: 0.5000
